In [1]:
import numpy as np
import stft
from scipy.io import wavfile
from scipy import signal
from matplotlib import pyplot as plt

In [2]:
%matplotlib inline

In [3]:
audio = wavfile.read('../audio_examples/piano_drums.wav')
sample_rate = audio[0]
audio = audio[1]
audio_l = audio[:, 0]
audio_r = audio[:, 1]

In [30]:
def welch(M):
    if M == 1:
        return np.ones(1, 'd')
    n = np.arange(0, M)
    w = 1 - ((n - (M-1.)/2.) / ((M-1.)/2.)) ** 2
    return w

In [4]:
def write_stereo(a_l, a_r, s_r, name):
    a_s = np.zeros([a_l.shape[0], 2], dtype=np.int16)
    a_s[:, 0] = a_l
    a_s[:, 1] = a_r
    print(a_s.shape)
    wavfile.write(name, s_r, a_s)

In [5]:
def write_mono(a_m, s_r, name):
    write_stereo(a_m, a_m, s_r, name)

In [6]:
def basic_stretch_linear(a, w=1.):
    """
    The weight parameter w does not really affect anything
    Doubles the array, filling the missing elements with a linear mix of consecutive samples:
    out[0] = input[0], out[1] = w * input[0] + (1 - w) * input[1], ...,
    
    w=1 corresponds to just doubling the array, the last sample is not repeated twice
    out[0] = input[0], out[1] = input[0], out[2] = input[1], ...
    
    w=0 corresponds to just doubling the array, the first sample is not repeated twice
    out[0] = input[0], out[1] = input[1], out[2] = input[2], ...
    """
    b = np.zeros(a.shape[0] * 2 - 1)
    b[::2] = a[:]
    b[1::2] = w * a[:-1] + (1-w) * a[1:]
    return b

In [21]:
def stft_stretch_basic(a, overlap, framelength, window_f,
                       w=1.,
                       i_overlap=None, i_window_f=None, keep_phase=True):
    """
    The weight parameter w does not really affect anything
    Perform a stft with (overlap, framelength, window_f), double the array,
    filling the missing elements with a linear mix of consecutive samples
    Then perform an istft with (i_overlap, framelength, i_window_f)
    if a parameter of the istft is None, the stft parameter will be used in its place
    if keep_phase = False, the phase data will be replaced with random data
    """
    if i_overlap is None:
        i_overlap = overlap
    if i_window_f is None:
        i_window_f = window_f
    
    sp = stft.spectrogram(a, framelength=framelength, window=window_f, overlap=overlap)
    out_sp = np.zeros((sp.shape[0], sp.shape[1] * 2 - 1), dtype=np.complex_)  # freq_bins * time_bins
    out_sp[:, ::2] = sp
    out_sp[:, 1::2] = w * sp[:, :-1] + (1-2) * sp[:, 1:]
    if not keep_phase:
        phase = np.random.rand(out_sp.shape[0], out_sp.shape[1]) * 2 * 3.141
        out_sp.imag = phase
    return stft.ispectrogram(out_sp, framelength=framelength, window=i_window_f, overlap=i_overlap)

In [46]:
def stft_paulstretch(a, overlap, framelength, stretch, window_f=welch):
    """
    Implements the Paul strech algorithm, as described here:
    http://www.paulnasca.com/algorithms-created-by-me#TOC-PaulStretch-extreme-sound-stretching-algorithm
    
    """
    sp = stft.spectrogram(a, framelength=framelength, window=window_f, overlap=overlap)
    out_sp = np.zeros((sp.shape[0], sp.shape[1]), dtype=np.complex_) 
    out_sp[:, :] = sp
    phase = np.random.rand(out_sp.shape[0], out_sp.shape[1]) * 2 * 3.141
    out_sp.imag = phase
    i_ov = overlap//stretch
    return stft.ispectrogram(out_sp, framelength=framelength, window=window_f, overlap=i_ov)

In [22]:
def make_stft_name(overlap, framelength, w_f_n, w=1,
                   i_overlap=None, i_w_f_n=None, keep_phase=True):
    name = '../audio_examples/p_d_'
    name += 'o' + str(overlap) + '_'
    name += 'f' + str(framelength) + '_'
    name += w_f_n + '_'
    name += 'w' + str(w) + '_'
    if i_overlap is not None:
        name += 'io' + str(i_overlap) + '_'
    if i_w_f_n is not None:
        name += i_w_f_n + '_'
    if not keep_phase:
        name += '_randphase'
    name += '.wav'
    return name

In [35]:
def make_paulname(overlap, framelength, stretch, w_f_n):
    name = '../audio_examples/p_d_'
    name += 'o' + str(overlap) + '_'
    name += 'f' + str(framelength) + '_'
    name += 's' + str(stretch) + '_'
    name += w_f_n + ''
    name += '.wav'
    return name

In [ ]:
# let's test the basic, pitch-shifting stretch function
write_stereo(basic_stretch_linear(audio_l),
             basic_stretch_linear(audio_r), sample_rate, '../audio_examples/piano_drum_basic_lin_1.wav')

In [23]:
# let's test the stft-based stretch function
overlap = 4
framelength = 2048
window_f = signal.cosine
w_f_n = "cosine"
w = 1  # this does not really affect anything
keep_phase = False

name = make_stft_name(overlap, framelength, w_f_n, w, keep_phase=keep_phase)
write_stereo(stft_stretch_basic(audio_l, overlap, framelength, window_f, w, keep_phase=keep_phase),
             stft_stretch_basic(audio_r, overlap, framelength, window_f, 1., keep_phase=keep_phase), sample_rate, name)

(610304, 2)


In [50]:
# some stereo-expansion can be achieved by using different window functions for the istft
overlap = 4
framelength = 2048
window_f = signal.hann
w_f_n = "cosine"
w = 1  # this does not really affect anything
keep_phase = False

name = make_stft_name(overlap, framelength, w_f_n, w, keep_phase=keep_phase)
write_stereo(stft_stretch_basic(audio_l, overlap, framelength, welch, 1., keep_phase=keep_phase),
             stft_stretch_basic(audio_r, overlap, framelength, signal.cosine, 1., keep_phase=keep_phase),
             sample_rate, name)

(610304, 2)


In [49]:
#let's test the Paul stretch algorithm
overlap = 16
framelength = 2048
stretch = 8

name = make_paulname(overlap, framelength, stretch, 'welch')

write_stereo(stft_paulstretch(audio_l, overlap, framelength, stretch, welch),
             stft_paulstretch(audio_l, overlap, framelength, stretch, welch),
             sample_rate, name)

(2441216, 2)
